In [1]:
import pandas as pd
import numpy as np
import re
import copy
import json
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

from util.api import create_lts_obj, get_pkgs_usage_count, get_packages_out_of_stackage, get_pkgs_out_transitively, get_all_time_packages, build_mtl_continuity_matrix, get_versions_df, get_update_count_df, get_count_updated_packages_by_lts, compute_monad_usage_by_df, get_added_packages_mtl_by_lts, get_removed_packages_mtl_by_lts, get_packages_started_use_mtl, get_packages_stopped_use_mtl

%matplotlib inline
plt.style.use('classic')

In [2]:
lts_list = ['0-7', '2-22', '3-22', '6-35', '7-24', '9-21', '11-22', '12-14', '12-26', '13-11', '13-19', '14-27', '15-3', '16-11']
df_list = []
df_by_category_list = []

for idx, lts in enumerate(lts_list):
    df = pd.read_pickle(f'../data/lts-{lts}/lts-{lts}.df')
    df_list.append(df)
    
for idx, lts in enumerate(lts_list):
    df = pd.read_pickle(f'../data/lts-{lts}/lts-{lts}-by-category.df')
    df_by_category_list.append(df)

In [16]:
df = df_list[0]
len(df[df['Control.Monad.Reader'] == 1].index)

52

In [14]:
from operator import itemgetter

df = df_list[0]
cont_df = df[df['Control.Monad.Reader'] == 1] # packages that use continuation monad
files_info_df = cont_df['files-info']

In [15]:
for files_info in files_info_df:
    files_info_list = re.findall(r'(?=\{).+?(?<=\})', files_info)

    for file_info in files_info_list:
        dictionary = json.loads(file_info)
        (imports, calls, file) = itemgetter('imports', 'calls', 'file')(dictionary)
        if 'Control.Monad.Reader' in imports:    
            print(calls, file)
            print()

['liftM', 'singleton', 'liftM', 'singleton'] /Users/feliperuiz/Desktop/github/tesis/data/lts-0.7/Chart/Chart-1.3.2/Graphics/Rendering/Chart/Backend/Impl.hs

['function', 'Seq.foldr', 'Seq.foldr', "lift'", '(.)', "lift'", '(.)', "lift'", '($)', 'transform', "frequency'", 'return', 'transform', '($)', "frequency'", '(:)', 'return'] /Users/feliperuiz/Desktop/github/tesis/data/lts-0.7/ChasingBottoms/ChasingBottoms-1.3.0.8/Test/ChasingBottoms/ContinuousFunctions.hs

[] /Users/feliperuiz/Desktop/github/tesis/data/lts-0.7/MaybeT/MaybeT-0.1.2/Control/Monad/Maybe.hs

['(.)', '(.)', '(.)', 'fromList', 'fmap', 'flip'] /Users/feliperuiz/Desktop/github/tesis/data/lts-0.7/MonadRandom/MonadRandom-0.3.0.1/Control/Monad/Random.hs

['asks', 'header', 'Nothing', 'return', 'Just', '($)', 'return', 'detectPreferredFormat', 'parseAll', '($)', 'return', 'A.decodeStrict', "A.eitherDecodeStrict'", 'Left', 'setStatus', 'text', '($)', 'T.pack', '(++)', 'Right', 'return', 'asks', 'asks', 'asks', 'return', '(++)',